# Qdrant

In [1]:
from langchain_qdrant import QdrantVectorStore
from qdrant_client import QdrantClient
from qdrant_client.http.models import VectorParams, Distance

from langchain_ollama import OllamaEmbeddings

from langchain_community.document_loaders import CSVLoader

embeddings = OllamaEmbeddings(model="nomic-embed-text", base_url="http://localhost:11434")

# client = QdrantClient(":memory:")
client = QdrantClient("http://localhost:6333")
collection_name = "test"
if not client.collection_exists(collection_name):
    client.create_collection("test", vectors_config=VectorParams(size=768, distance=Distance.COSINE))
vector_store = QdrantVectorStore(
    client=client,
    collection_name="test",
    embedding=embeddings,
)

def load_data():
    file_path = "./data/mlb_teams_2012.csv"
    loader = CSVLoader(file_path, source_column="Team")
    return loader.load()

documents = load_data()
ids = vector_store.add_documents(documents)
print(ids)

['f69119d4b20347d493ec25c5f8aedf4f', '937b46e414f14fa2993007534e7d2650', '49b54d3a3ec84305903da5762399c87b', '12f72b849c2b401b804770d38a27c14e', '9512bd368e5642cc9e23386e996a6b64', 'cb2a61b75ce940629768c752f9add748', 'b6b332e6d43e451689dfb2da727d1384', 'd88b8731ba1a4ece810a51841035ab27', 'b5c092b0b15647f79665ce3d00b87c40', '37e0fc50f79b489393297286ce12e7e7', '2dadd74640574ac981b8d301ee6ad285', '69a9dea11cad40dfa609a297eae41892', 'dc53e2d7ca72443dabe3f679c644f71c', '81be11121691430ba58556aa3094f6c4', '331900952f8640ab8b076402f063960e', 'aad19e3ed7f640e89fc6af860044f78b', '40ac50dbce2d4e64a98543b857788b5f', '1060161c8405444c8292bee600a494fb', 'f67e09e4a8b641028502afe183906a4a', '2a87b2711fd14fc49201a5fc7c47ef66', '99aefef4564645cebc58bf127a85109a', '9bbb7e8d393a47b287ee644b6cb076b1', 'c6343edbb82a4ce29912178faf7f9fda', '223239688eb1488ba9c5d72dd982628c', '731f82fa8fae49e4ae845cd1134db991', '4a7c14ed6f524618bb65704ad823c06c', 'd358700976084bd99dfcc810cc0795cb', '1bab4d546ba64c298dc9ef2624

In [3]:
from pprint import pprint
# client = QdrantClient("http://localhost:6333")
# client = QdrantClient(host="localhost", port=6333)
client = QdrantClient(host="localhost", grpc_port=6334)
collection_info = client.get_collection(collection_name)
pprint(collection_info)

# ids = vector_store.add_documents(documents)
# collection_info = client.get_collection(collection_name)
# print(collection_info)

CollectionInfo(status=<CollectionStatus.GREEN: 'green'>, optimizer_status=<OptimizersStatusOneOf.OK: 'ok'>, vectors_count=None, indexed_vectors_count=0, points_count=30, segments_count=8, config=CollectionConfig(params=CollectionParams(vectors=VectorParams(size=768, distance=<Distance.COSINE: 'Cosine'>, hnsw_config=None, quantization_config=None, on_disk=None, datatype=None, multivector_config=None), shard_number=1, sharding_method=None, replication_factor=1, write_consistency_factor=1, read_fan_out_factor=None, on_disk_payload=True, sparse_vectors=None), hnsw_config=HnswConfig(m=16, ef_construct=100, full_scan_threshold=10000, max_indexing_threads=0, on_disk=False, payload_m=None), optimizer_config=OptimizersConfig(deleted_threshold=0.2, vacuum_min_vector_number=1000, default_segment_number=0, max_segment_size=None, memmap_threshold=None, indexing_threshold=20000, flush_interval_sec=5, max_optimization_threads=None), wal_config=WalConfig(wal_capacity_mb=32, wal_segments_ahead=0), quan